In [7]:
import chromadb
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
import dotenv
import openai
import os

dotenv.load_dotenv("../.env", override=True)

True

In [8]:
def get_openai_embedding(text: str):
    """Generate embeddings using OpenAI's API (new version)."""
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding


In [9]:
def load_and_split_docs(directory: str, chunk_size: int = 1000, chunk_overlap: int = 200):
    """
    Loads all .txt files from the given directory and splits them into smaller chunks
    for embedding. Returns a list of LangChain Documents.
    """
    loader = DirectoryLoader(directory, glob="*.txt")
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    documents = text_splitter.split_documents(docs)
    return documents


In [10]:
docs = load_and_split_docs(directory="../txts")

In [11]:
def create_or_load_vectorstore(persist_directory: str, documents=None):
    """
    If 'documents' are provided, create a new Chroma store from them and persist it.
    Otherwise, load from the existing persist_directory.
    """
    embedding_function = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
    
    # Create/update the store from the documents
    vectorstore = Chroma.from_documents(
        documents,
        embedding_function,
        collection_name="documents",
        persist_directory=persist_directory
    )
    vectorstore.persist()
    return vectorstore


In [ ]:
vectorstore = create_or_load_vectorstore(persist_directory="../chromadb",
                           documents=docs)

In [19]:
def retrieve_best_chunks(text: str, n: int, db_path: str = "../chromadb"):
    """
    Retrieve the best N chunks from a local ChromaDB database based on similarity to the input text.
    
    :param text: The query text to search for similar chunks.
    :param n: The number of best matching chunks to retrieve.
    :param db_path: Path to the local ChromaDB database.
    :return: A list of tuples containing (chunk, score).
    """
    # Generate query embedding using OpenAI
    query_embedding = get_openai_embedding(text)
    
    # Connect to ChromaDB
    client = chromadb.PersistentClient(path=db_path)
    collection = client.get_collection(name="documents")
    
    # Perform similarity search
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n
    )
    
    # Extract chunks and scores
    chunks = results["documents"][0] if "documents" in results else []
    scores = results["distances"][0] if "distances" in results else []
    
    return list(zip(chunks, scores))

In [21]:
openai.api_key = os.getenv("OPENAI_API_KEY")  # Ensure API key is set in environment variables
query_text = "l Sole in quadratura con Urano in Toro"
best_chunks = retrieve_best_chunks(query_text, n=5)
for chunk, score in best_chunks:
    print(f"Chunk: {chunk}\nScore: {score}\n")

Chunk: emotive e di costruire fondamenta più solide nella tua vita personale e professionale.

Per il tuo compagno, il transito di Giove e Urano in Toro influisce sulla sua seconda casa solare (se

consideriamo i Gemelli come segno ascendente di base). Questo porta attenzione alla stabilità finanziaria

e ai valori personali. Potrebbe cercare di migliorare la sua sicurezza economica e il senso di autostima.

Temi karmici Un tema importante per te è il perdono: potresti portare con te ferite del passato (anche in

vite precedenti) legate a tradimenti o mancanza di riconoscimento. Per il tuo compagno, la lezione karmica

riguarda la capacità di impegnarsi pienamente, trovando un equilibrio tra libertà personale e dedizione

verso gli altri.

Domande

Hai sollevato tre domande principali:

1. Problemi sentimentali: possono essere risolti?

Analizzerò le dinamiche della relazione per comprendere eventuali ostacoli e proporre soluzioni

per superarli.

2. Figlio in arrivo: è previsto e quan